In [1]:
import sys; sys.path.insert(0, "..")
import re
import operator
from functools import reduce

import pandas as pd
import numpy as np
import deba

deba.set_root('..')

from features.minutes import extract_features, REGEXES_MAP

In [2]:
df = extract_features()
df

,fileid,pageno,filetype,region,year,month,day,file_category,filepath,filesha1,text,mtg,hrg,agd
0,d4c619b,1,pdf,addis,NaN,NaN,NaN,minutes,addis/atc/120518MeetingMinutes.pdf,d4c619b9d7d91a3d83fc4d374fe679fe6ca970a6,"[TOWN OF ADDIS MINUTES, REGULAR MEETING-DECEMB...",True,False,False
1,d4c619b,2,pdf,addis,NaN,NaN,NaN,minutes,addis/atc/120518MeetingMinutes.pdf,d4c619b9d7d91a3d83fc4d374fe679fe6ca970a6,"[Town of Addis Minutes, Regular Meeting — Dece...",True,False,False
2,d4c619b,3,pdf,addis,NaN,NaN,NaN,minutes,addis/atc/120518MeetingMinutes.pdf,d4c619b9d7d91a3d83fc4d374fe679fe6ca970a6,"[Town of Addis Minutes, Regular Meeting- Decem...",True,False,False
3,d4c619b,4,pdf,addis,NaN,NaN,NaN,minutes,addis/atc/120518MeetingMinutes.pdf,d4c619b9d7d91a3d83fc4d374fe679fe6ca970a6,"[ANY OTHER BUSINESS, Mayor Toups presented the...",False,False,False
4,6bce918,1,pdf,addis,2020.0,5.0,6.0,minutes,addis/atc/AI4-MINUTES-2020-05-06.pdf,6bce918f42fab13a26e15350a3f67ed6357de899,"[TOWN OF ADDIS MINUTES, May 6, 2020 — Regular ...",True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3525,3cc0f60,1,pdf,youngsville,2020.0,10.0,28.0,minutes,youngsville/ycsb/2020-10-28 YPCSB Minutes.pdf,3cc0f6043caf6072d0b1000358af174eb7748d52,[MINUTES OF THE REGULAR MEETING OF THE YOUNGSV...,True,False,False
3526,3cc0f60,2,pdf,youngsville,2020.0,10.0,28.0,minutes,youngsville/ycsb/2020-10-28 YPCSB Minutes.pdf,3cc0f6043caf6072d0b1000358af174eb7748d52,"[AGENDA ITEM 5. Adjourn, A motion was offered ...",False,False,False
3527,09982eb,1,pdf,youngsville,2020.0,11.0,17.0,minutes,youngsville/ycsb/2020-11-17 Special Teleconfe...,09982eb6ffef305c9aa1fbd1cf2f1bc3bb6194bc,[MINUTES OF THE SPECIAL TELECONFERENCE MEETING...,True,False,False
3528,5fe4f2a,1,pdf,youngsville,2020.0,12.0,15.0,minutes,youngsville/ycsb/2020-12-15 YPCSB Minutes.pdf,5fe4f2a4d5859c451e81e3681c7b537fac86d51b,[MINUTES OF THE SPECIAL MEETING OF THE YOUNGS ...,True,False,False


In [3]:
page_cls = df[["fileid", "pageno", "lineno", "text"]].set_index(["fileid", "pageno", "lineno"], drop=False)
page_cls.loc[:, "hd"] = page_cls.lineno <= 3
linenos = df[["fileid", "pageno", "lineno"]].groupby(["fileid", "pageno"]).max().join(page_cls.lineno.to_frame(), how='right', lsuffix='_max')
page_cls.loc[:, "ft"] = linenos.lineno == linenos.lineno_max

page_cls.loc[:, "re_frontpage_1"] = ((page_cls.hd | page_cls.ft) & page_cls.text.str.match(r"page 1$", flags=re.IGNORECASE)).astype(int)
page_cls.loc[:, "re_frontpage_2"] = (page_cls.ft & page_cls.text.str.match(r"^1$", flags=re.IGNORECASE)).astype(int)
page_cls.loc[:, "re_contpage_1"] = ((page_cls.hd | page_cls.ft) & page_cls.text.str.match(r"page ([2-9]|1[0-9])", flags=re.IGNORECASE)).astype(int)
page_cls.loc[:, "re_contpage_2"] = (page_cls.ft & page_cls.text.str.match(r"^([2-9]|1[0-9])$", flags=re.IGNORECASE)).astype(int)

page_cls = page_cls.reset_index(drop=True)
page_cls = page_cls[['fileid', 'pageno']+[col for col in page_cls.columns if col.startswith("re")]].groupby(['fileid', 'pageno']).max()
page_cls
# df[["fileid", "pageno", "lineno"]].groupby(["fileid", "pageno"]).max().reset_index()

KeyError: "['lineno'] not in index"

In [ ]:
hdr_fts = df.loc[df.lineno <= 8, ["region", "fileid", "pageno", "lineno", "text"]].set_index(['fileid','pageno','lineno'])
def match(group:pd.DataFrame):
    region = group.region[0]
    if region not in REGEXES_MAP:
        return group
    for feat, patterns in REGEXES_MAP[region].items():
        group.loc[:, feat] = reduce(operator.or_, [group.text.str.match(pattern) for pattern in patterns], False)
    return group
hdr_fts = hdr_fts.groupby("region").apply(match)
feat_cols = ['mtg', 'hrg', 'agd']
hdr_fts.loc[:, feat_cols] = hdr_fts[feat_cols].fillna(False).astype(int)
hdr_fts = hdr_fts.reset_index(drop=False).groupby(['fileid', 'pageno'])[feat_cols].max()
hdr_fts
    # if region not in REGEXES_MAP:
    #     continue
    # for feat, patterns in REGEXES_MAP[region].items():
    #     for pattern in patterns:
    #         hdr_fts.loc[frame.text.str.match(pattern), feat] = 1
# hdr_fts

mtg  hrg  agd
fileid  pageno               
003b2bf 1         1    0    0
        2         0    0    0
        3         0    0    0
        4         0    0    0
00508a5 1         1    0    0
...             ...  ...  ...
ff43b3c 1         0    0    0
        2         0    0    0
ffe2b9b 1         1    0    0
        2         0    0    0
        3         0    0    0

[3530 rows x 3 columns]

In [ ]:
hdr_fts

mtg  hrg  agd
fileid  pageno               
003b2bf 1         1    0    0
        2         0    0    0
        3         0    0    0
        4         0    0    0
00508a5 1         1    0    0
...             ...  ...  ...
ff43b3c 1         0    0    0
        2         0    0    0
ffe2b9b 1         1    0    0
        2         0    0    0
        3         0    0    0

[3530 rows x 3 columns]

In [ ]:
linenos

lineno_max  lineno
fileid  pageno lineno                    
003b2bf 1      0               34       0
               1               34       1
               2               34       2
               3               34       3
               4               34       4
...                           ...     ...
ffe2b9b 3      3                7       3
               4                7       4
               5                7       5
               6                7       6
               7                7       7

[125855 rows x 2 columns]